In [ ]:
from datetime import date, datetime, time
import pandas as pd
import pytz
import numpy as np
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetCalendarRequest
from alpaca.trading.models import Calendar
from alpaca.data import (
    CryptoHistoricalDataClient,
    StockHistoricalDataClient,
    StockSnapshotRequest,
    StockLatestBarRequest,
    CryptoBarsRequest,
    StockBarsRequest
)
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from liualgotrader.common import config
from typing import Tuple
from zoneinfo import ZoneInfo

In [ ]:
NY = "America/New_York"
nytz = pytz.timezone(NY)

In [ ]:
trading_client = TradingClient(
    api_key=config.alpaca_api_key,
    secret_key=config.alpaca_api_secret,
    paper=True,
)
stock_client = StockHistoricalDataClient(
    api_key=config.alpaca_api_key,
    secret_key=config.alpaca_api_secret,)
crypto_client = CryptoHistoricalDataClient(
    api_key=config.alpaca_api_key,
    secret_key=config.alpaca_api_secret,)

### Calendar

In [ ]:
trading_days = trading_client.get_calendar(  
            filters=GetCalendarRequest(start='2023-01-01', end='2023-02-01')
        )

In [ ]:
trading_days

In [ ]:
trading_days[0].open.time()

### Assets

In [ ]:
trading_client.get_asset("ADDYY")

### snapshots

In [ ]:
data = stock_client.get_stock_snapshot(StockSnapshotRequest(symbol_or_symbols="ADDYY"))

In [ ]:
data

### Crypto


In [ ]:
start=datetime.combine(date(year=2021, month=4, day=1), datetime.min.time())
end=datetime.combine(date(year=2021, month=4, day=2), datetime.min.time())
print(start, end)

In [ ]:
bars = crypto_client.get_crypto_bars(
    CryptoBarsRequest(
        start=start,
        end=end,
        timeframe=TimeFrame(1, TimeFrameUnit.Minute),
        symbol_or_symbols=["BTC/USD"])).df
bars

In [ ]:
start=datetime.combine(date(year=2021, month=4, day=1), datetime.min.time())
bars = crypto_client.get_crypto_bars(
    CryptoBarsRequest(
        start=start,
        timeframe=TimeFrame(1, TimeFrameUnit.Minute),
        symbol_or_symbols=["BTC/USD"])).df
bars

### Equity

In [ ]:
bars = stock_client.get_stock_latest_bar(
    StockLatestBarRequest(
        timeframe=TimeFrame(1, TimeFrameUnit.Minute),
        symbol_or_symbols="aapl"))

bars['AAPL'].timestamp.astimezone(tz=nytz)

In [ ]:
start=datetime.combine(date(year=2020, month=10, day=1), time(hour=0, minute=30), tzinfo=ZoneInfo("US/Eastern"))
end=datetime.combine(date(year=2020, month=10, day=2), time(hour=16, minute=0),tzinfo=ZoneInfo("US/Eastern"))
bars = stock_client.get_stock_bars(
    StockBarsRequest(
        start=start,
        end=end,
        timeframe=TimeFrame(1, TimeFrameUnit.Day),
        symbol_or_symbols="aapl")).df
bars = bars.reset_index(level=0, drop=True)
bars = bars.tz_convert("America/New_York")
bars

In [ ]:
start=datetime.combine(date(year=2021, month=4, day=1), datetime.min.time())
end=datetime.combine(date(year=2021, month=4, day=2), datetime.min.time())
print(start, end)

In [ ]:
bars = stock_client.get_stock_bars(
    StockBarsRequest(
        start=start,
        end=end,
        timeframe=TimeFrame(1, TimeFrameUnit.Minute),
        symbol_or_symbols="aapl")).df
bars

In [ ]:
bars = bars.reset_index(level=0, drop=True)
bars

In [ ]:
bars = bars.tz_convert("America/New_York")

In [ ]:
bars

In [ ]:
bars.loc['2021-04-01 05:02:00-04:00']

### multi symbol bars

In [ ]:
bars = stock_client.get_stock_bars(
    CryptoBarsRequest(
        start=start,
        end=end,
        timeframe=TimeFrame(1, TimeFrameUnit.Minute),
        symbol_or_symbols=["aapl", "ibm"])).df

In [ ]:
bars

In [ ]:
bars.rename(columns={"vwap": "average"}, inplace=True)
bars.rename(columns={"trade_count": "count"}, inplace=True)
bars["vwap"] = np.NaN

bars